<h1 align="center" style="color:#96d9f0;">Computational Intelligence for
Optimization - Project</h1>
<h3 align="center" style="color:#96d9f0;">Group AW - Wedding Seating Optimization</h3>

---

### <span style="color:#96d9f0;">Group Members</span>

<table>
  <thead style="color:#96d9f0;">
    <tr>
      <th>Name</th>
      <th>Email</th>
      <th>Student ID</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Afonso Dias</td>
      <td>20211540@novaims.unl.pt</td>
      <td>20211540</td>
    </tr>
    <tr>
      <td>Isabel Duarte</td>
      <td>20240545@novaims.unl.pt</td>
      <td>20240545</td>
    </tr>
    <tr>
      <td>Pedro Campino</td>
      <td>20240537@novaims.unl.pt</td>
      <td>20240537</td>
    </tr>
    <tr>
      <td>Rita Matos</td>
      <td>20211642@novaims.unl.pt</td>
      <td>20211642</td>
    </tr>
  </tbody>
</table>

In [ ]:
import sys
sys.path.append('..')

: 

In [ ]:
import random
from copy import deepcopy
from library.solution import Solution
import csv
import os
import pandas as pd

: 

In [ ]:
#No swap mutation within the same table

: 

In [ ]:
# Import csv
import csv
import os
import pandas as pd

relationship_matrix = pd.read_csv('seating_data.csv')

: 

In [ ]:
from library.problems.data.tsp_data import distance_matrix

class WSSolution(Solution):
    def __init__(self, repr=None, distance_matrix=distance_matrix, starting_idx=0):
        self.starting_idx = starting_idx
        self.distance_matrix = distance_matrix
        super().__init__(repr=repr)

: 